In [1]:
import circuitsvis as cv
import numpy as np
import torch
from taker import Model
from taker.hooks import HookConfig
from datetime import datetime
import json
from os import listdir
from os.path import exists

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import torch
import csv

/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
m = Model("google/gemma-2-9b-it", dtype="bfp16")
# m = Model("google/gemma-2-2b-it")
m.show_details()

/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  8.14it/s]


Loaded model 'google/gemma-2-9b-it' with bfp16:
- Added 672 hooks across 42 layers
 - n_layers : 42
 - d_model  : 3584
 - n_heads  : 16
 - d_head   : 256
 - d_mlp    : 14336


In [3]:
with open('../promptsV1.csv', newline='') as f:
    reader = csv.reader(f)
    readdata = list(reader)
    readdata = readdata[:20]

In [4]:
import sys, os
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [5]:
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


In [9]:
# filename = f"../gemma_results/latest_orig_generation.jsonl"
# if not exists(filename):
#     with open(filename, "w") as f:
#         pass

# max_new_tokens = 200
# temperature = 0.3

# [h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
# for prompt in readdata:
#     prompt = prompt[0]
#     with HiddenPrints():
#         for i in range(10):
#             output = m.generate(prompt, max_new_tokens, temperature=temperature)
#             print(output)
#             data = {
#                 "temperature": temperature,
#                 "max_new_tokens": max_new_tokens,
#                 "model": "google/gemma-2-2b-it",
#                 "type": "original",
#                 "transplant_layers": None,
#                 "prompt": prompt,
#                 "output": output[1],
#             }

#             with open(filename, "a") as file:
#                 file.write(json.dumps(data) + "\n")

# filename = f"../gemma_results/latest_neutral_generation_V2.jsonl"
# if not exists(filename):
#     with open(filename, "w") as f:
#         pass

# neutral_prompts = ["\n\n"]
# # for neutral in neutral_prompts:                
# #     with HiddenPrints():
# #         for i in range(50):
# #             output = m.generate(neutral, max_new_tokens, temperature=temperature)
            
# #             data = {
# #                 "temperature": temperature,
# #                 "max_new_tokens": max_new_tokens,
# #                 "model": "google/gemma-2-2b-it",
# #                 "type": "neutral",
# #                 "transplant_layers": None,
# #                 "prompt": neutral,
# #                 "output": output[1],
# #             }

# #             with open(filename, "a") as file:
# #                 file.write(json.dumps(data) + "\n")



# orig_df = pd.read_json(f"../gemma_results/latest_orig_generation.jsonl", lines=True)
# def split_at_double_newline(text):
#     # Ensure we are only working with strings longer than 15 characters
#     if len(text) > 15:
#         # Search for the first double newline after the 15th character
#         pos = text.find('\n\n', 15)
#         if pos != -1:  # Check if double newline was found
#             return text[:pos+2], text[pos:]  # Split and remove the newline from the second part
#     return text, None  # If no split is required, return the original text and None

# # Apply the function to the DataFrame column
# orig_df['paragraph1'], orig_df['paragraph2'] = zip(*orig_df['output'].apply(split_at_double_newline))
# orig_df['paragraph1'] = orig_df['prompt'].astype(str) + orig_df['paragraph1'].astype(str)
# print(repr(orig_df['paragraph1'][0]))
# filename = f"../gemma_results/latest_transferred_generation_V2.jsonl"
# if not exists(filename):
#     with open(filename, "w") as f:
#         pass
    
# for info_prompt in orig_df['paragraph1']:
#     acts = m.get_midlayer_activations(info_prompt)
#     orig_token_index = m.get_ids(info_prompt).shape[1] - 1
#     for neutral_prompt in neutral_prompts:
#         new_token_index  = m.get_ids(neutral_prompt).shape[1] - 1        

#         [h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
#         for layer_index in range(0,26):
#             m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index, acts["mlp"][0, layer_index, orig_token_index]*100)
#             m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index, acts["attn"][0, layer_index, orig_token_index]*100)
#         with HiddenPrints():
#             # for i in range(1):
#                 output = m.generate(neutral_prompt, max_new_tokens, temperature=temperature)
                
#                 data = {
#                     "temperature": temperature,
#                     "max_new_tokens": max_new_tokens,
#                     "model": "google/gemma-2-2b-it",
#                     "type": "transferred",
#                     "num_transferred_tokens": 2,
#                     "transplant_layers": (0,26),
#                     "orig_prompt": info_prompt,
#                     "transplant_prompt": neutral_prompt,
#                     "output": output[1],
#                 }

#                 with open(filename, "a") as file:
#                     file.write(json.dumps(data) + "\n")

'Tell me about a weekend in a mountain cabin in 150 words and then tell me about the Saint Louis Arch in another 150 words. Start the second paragraph with "On a lovely Sunday morning the roadways are bustling"\n\nOn a crisp autumn weekend, I found myself nestled in a cozy mountain cabin. The air was crisp and clean, the leaves were ablaze in fiery hues of red and gold, and the silence was broken only by the crackling fire in the stone fireplace. I spent my time reading by the window, sipping hot cocoa, and watching the snow softly fall. The cabin was filled with the scent of pine needles and woodsmoke, and the warmth of the fire radiated through the room. It was a perfect escape from the hustle and bustle of city life.\n\n'


# V2 with Cheating Neutral Generation

In [5]:
filename = f"../gemma9b_results/latest_orig_generation_new.jsonl"
if not exists(filename):
    with open(filename, "w") as f:
        pass

max_new_tokens = 200
temperature = 0.3

# [h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
# for prompt in readdata:
#     prompt = prompt[0]
#     with HiddenPrints():
#         for i in range(50):
#             output = m.generate(prompt, max_new_tokens, temperature=temperature)
#             print(output)
#             data = {
#                 "temperature": temperature,
#                 "max_new_tokens": max_new_tokens,
#                 "model": "google/gemma-2-9b-it",
#                 "type": "original",
#                 "transplant_layers": None,
#                 "prompt": prompt,
#                 "output": output[1],
#             }

#             with open(filename, "a") as file:
#                 file.write(json.dumps(data) + "\n")

orig_df = pd.read_json(f"../gemma9b_results/latest_orig_generation_new.jsonl", lines=True)
def split_at_double_newline(text):
    # Ensure we are only working with strings longer than 15 characters
    if len(text) > 15:
        # Search for the first double newline after the 15th character
        pos = text.find('\n\n', 15)
        if pos != -1:  # Check if double newline was found
            return text[:pos+2], text[pos:]  # Split and remove the newline from the second part
    return text, text  # If no split is required, return the original text and None

orig_df['paragraph1'], orig_df['paragraph2'] = zip(*orig_df['output'].apply(split_at_double_newline))
orig_df['paragraph1'] = orig_df['prompt'].astype(str) + orig_df['paragraph1'].astype(str)
print(repr(orig_df['paragraph1'][0]))
def get_neutral_prompt(text):
    idlist = m.get_ids(text).squeeze().tolist()
    neutral_tokens = m.tokenizer.convert_ids_to_tokens(idlist)
    neutral_tokens = [entry.replace("▁", " ") for entry in neutral_tokens]
    sep = ''
    return sep.join(neutral_tokens[1:3]), sep.join(neutral_tokens[1:4]), sep.join(neutral_tokens[1:5]), sep.join(neutral_tokens[1:6]), sep.join(neutral_tokens[1:7])
print(repr(orig_df['paragraph2'][0]))
orig_df["neutral_prompt1"], orig_df["neutral_prompt2"], orig_df["neutral_prompt3"], orig_df["neutral_prompt4"], orig_df["neutral_prompt5"] = zip(*orig_df['paragraph2'].apply(get_neutral_prompt))
print(orig_df["neutral_prompt1"][0])
print(orig_df["neutral_prompt2"][0])
print(orig_df["neutral_prompt3"][0])
print(orig_df["neutral_prompt4"][0])
print(orig_df["neutral_prompt5"][0])

[h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS


for neutralnum in range(1, 6):
    filename = f"../gemma9b_results/latest_neutral{neutralnum}_generation.jsonl"
    if not exists(filename):
        with open(filename, "w") as f:
            pass

    for neutral in orig_df[f"neutral_prompt{neutralnum}"]:                
        with HiddenPrints():
            output = m.generate(neutral, max_new_tokens, temperature=temperature)
            
            data = {
                "temperature": temperature,
                "max_new_tokens": max_new_tokens,
                "model": "google/gemma-2-9b-it",
                "type": "neutral",
                "cheat_tokens": neutralnum,
                "transplant_layers": None,
                "prompt": neutral,
                "output": output[1],
            }

            with open(filename, "a") as file:
                file.write(json.dumps(data) + "\n")



neutral_prompts = ["\n\n"]
# Apply the function to the DataFrame column
filename = f"../gemma9b_results/latest_transferred_generation.jsonl"
if not exists(filename):
    with open(filename, "w") as f:
        pass
    
for info_prompt in orig_df['paragraph1']:
    acts = m.get_midlayer_activations(info_prompt)
    orig_token_index = m.get_ids(info_prompt).shape[1] - 1
    for neutral_prompt in neutral_prompts:
        new_token_index  = m.get_ids(neutral_prompt).shape[1] - 1        

        [h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
        for layer_index in range(0,26):
            m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index, acts["mlp"][0, layer_index, orig_token_index])
            m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index, acts["attn"][0, layer_index, orig_token_index])
        with HiddenPrints():
            # for i in range(1):
                output = m.generate(neutral_prompt, max_new_tokens, temperature=temperature)
                
                data = {
                    "temperature": temperature,
                    "max_new_tokens": max_new_tokens,
                    "model": "google/gemma-2-9b-it",
                    "type": "transferred",
                    "num_transferred_tokens": 1,
                    "transplant_layers": (0,26),
                    "orig_prompt": info_prompt,
                    "transplant_prompt": neutral_prompt,
                    "output": output[1],
                }

                with open(filename, "a") as file:
                    file.write(json.dumps(data) + "\n")

'Tell me about a weekend in a mountain cabin in 150 words and then tell me about disconnecting from technology in another 150 words. Only do that. Make sure you don’t add any headings or comments.\n\nThe scent of pine needles filled the air as we drove up the winding mountain road.  Our cozy cabin, nestled amongst towering trees, welcomed us with warmth and the promise of a peaceful escape.  Days were spent hiking through sun-dappled forests, the sound of birdsong our only soundtrack. Evenings were spent by the crackling fireplace, sharing stories and laughter. The stars, unfiltered by city lights, blazed across the night sky, a breathtaking spectacle.  \n\n'
'\n\nThe world seemed to fade away as I stepped away from the constant buzz of notifications and the endless scroll.  Silence, once a rarity, became a comforting presence.  I rediscovered the joy of reading a physical book, the pages turning slowly, each word savored.  Conversations flowed more deeply, unhurried and genuine.  Natu

KeyboardInterrupt: 